In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import random
from statistics import mode
import re
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords, wordnet
from sklearn.feature_extraction.text import CountVectorizer
from gensim.corpora import Dictionary
from gensim import corpora, models
from gensim.models.ldamodel import LdaModel
%config InlineBackend.figure_format = 'retina'

In [2]:
df = pd.read_csv('sentiment_data/grand_df.csv', index_col=0)
df

,album,anger,anticipation,api_path,disgust,fear,joy,lyrics,negative,polarity,positive,primary_artist,release_date,sadness,song_endpoint,surprise,title,trust,url
0,Boy In Detention (Mixtape),14.0,13.0,/songs/62387,5.0,7.0,12.0,"Beats I be servin', Saks Fifth splurgin' Diam...",11.0,-0.9809,24.0,Chris Brown,2011-08-05,5.0,http://api.genius.com/songs/62387,11.0,100%,17.0,https://genius.com/Chris-brown-100-lyrics
1,Boy In Detention (Mixtape),19.0,1.0,/songs/54007,16.0,15.0,1.0,We in the motherfuckin' buildin' 100 fuckin...,20.0,-0.9967,13.0,Chris Brown,2011-08-05,16.0,http://api.genius.com/songs/54007,1.0,100 Bottles,4.0,https://genius.com/Chris-brown-100-bottles-lyrics
2,X,2.0,4.0,/songs/516349,2.0,1.0,6.0,You got me doing 101 on the 101 Don't go nowh...,6.0,0.9516,11.0,Chris Brown,2014-09-16,2.0,http://api.genius.com/songs/516349,2.0,101 (Interlude),6.0,https://genius.com/Chris-brown-101-interlude-l...
3,None,0.0,0.0,/songs/3103904,0.0,4.0,0.0,Ain't no way we could ignore this fire When I...,3.0,0.8592,4.0,Chris Brown,2017-05-31,2.0,http://api.genius.com/songs/3103904,0.0,10 Feet,2.0,https://genius.com/Chris-brown-10-feet-lyrics
4,None,20.0,17.0,/songs/57217,11.0,12.0,14.0,"Welcome to the Cypher Myself, K-Mac, Ace Hood...",23.0,0.9745,39.0,Chris Brown,2019-04-04,9.0,http://api.genius.com/songs/57217,17.0,2011 BET Cypher Chris Brown and Friends,25.0,https://genius.com/Chris-brown-2011-bet-cypher...
5,Fortune,3.0,11.0,/songs/79425,1.0,12.0,11.0,Faire l'amour lendemain What if the sky was...,8.0,0.8478,14.0,Chris Brown,2012-06-29,2.0,http://api.genius.com/songs/79425,2.0,2012,4.0,https://genius.com/Chris-brown-2012-lyrics
6,None,8.0,10.0,/songs/60609,8.0,11.0,14.0,Shattered glasses and cigarettes There's a lo...,37.0,-0.9576,13.0,Chris Brown,2012-06-19,9.0,http://api.genius.com/songs/60609,13.0,2 Complicated,13.0,https://genius.com/Chris-brown-2-complicated-l...
7,None,6.0,3.0,/songs/3089879,5.0,2.0,15.0,"Oh, oh, oh, babe, yeah I know what you need, ...",13.0,-0.1984,47.0,Chris Brown,2017-05-28,3.0,http://api.genius.com/songs/3089879,3.0,2 the Side,23.0,https://genius.com/Chris-brown-2-the-side-lyrics
8,None,16.0,7.0,/songs/2404874,16.0,15.0,4.0,And I'm like Jordan wearin' 45 I told you I w...,27.0,-0.9491,18.0,Chris Brown,2017-05-31,20.0,http://api.genius.com/songs/2404874,7.0,45,26.0,https://genius.com/Chris-brown-45-lyrics
9,Fan of a Fan,13.0,10.0,/songs/2877,9.0,9.0,7.0,"Yeah, leggo It's what they wanted right? Bl...",15.0,-0.7584,15.0,Chris Brown,2010-05-16,9.0,http://api.genius.com/songs/2877,7.0,48 Bar Rap,9.0,https://genius.com/Chris-brown-48-bar-rap-lyrics


In [8]:
count = df.groupby('primary_artist')[['title']].count().sort_values(by='title', ascending=False)

In [9]:
count[count.title > 200]

,title
primary_artist,
Gucci Mane,1026
Glee Cast,650
Chris Brown,588
Snoop Dogg,580
Wiz Khalifa,519
Young Thug,504
Elton John,464
Meek Mill,432
Eminem,418
